<h1>Goal Prediction Model</h1>
<h3>We want to build and train a model to predict goal from our data</h3>
<h3>What should we do?</h3>
<li>Import necessary libraries and load data</li>
<li>Data pre processing</li>
<li>Building models</li>

<h2>Import Necessary Libraries</h2>

In [1]:
import numpy as np
import pandas as pd

<h1>Data Pre_Processing</h1>

<h2>Load Data</h2>

In [2]:
df = pd.read_csv('dataset.csv')
print(df.shape)
df.head()

(8925, 13)


,Unnamed: 0,matchId,playerId,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,outcome
0,0,m_91,p_103,in game,right foot,13.47,-11.22,1,0,normal,70,9,goal
1,1,m_17,p_16,in game,left foot,9.48,14.22,3,0,normal,55,4,saved
2,2,m_111,p_88,free kick,left foot,29.43,-1.25,6,2,low,86,31,saved
3,3,m_142,p_87,in game,right foot,26.93,1.00,4,1,normal,77,2,lost
4,4,m_117,p_9,in game,right foot,10.72,5.24,2,0,normal,76,46,goal


<h2>Delete matchId, playerId and second</h2>
<h3>These features aren't effecient for our model so we drop it</h3>

In [3]:
df = df.drop(['matchId','playerId','Unnamed: 0','second'],axis=1)

<h2>Replace String Values With Int Values</h2>

In [4]:
items = ['others','head','right foot','left foot']
for ind,val in enumerate(items):
    df['bodyPart'][df['bodyPart'] == val ] = ind + 1

C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3828732416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bodyPart'][df['bodyPart'] == val ] = ind + 1


In [5]:
item = ['from corner','free kick','in game','penalty']
for ind,val in enumerate(item):
    df['playType'][df['playType'] == val] = ind + 1
    

C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3089727933.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['playType'][df['playType'] == val] = ind + 1


In [6]:
item = ['low','normal','much']
for ind, val in enumerate(item):
    df['interferenceOnShooter'][df['interferenceOnShooter'] == val] = ind + 1

C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3445012663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['interferenceOnShooter'][df['interferenceOnShooter'] == val] = ind + 1


In [7]:
item = ['defence blocked','own goal','lost','saved','off the post','goal']
for val in item:
    if 'goal' in val:
        df['outcome'][df['outcome'] == val] = 1
    else : 
        df['outcome'][df['outcome'] == val] = 0

C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\422814725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outcome'][df['outcome'] == val] = 0
C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\422814725.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outcome'][df['outcome'] == val] = 1


<h2>Calculate distance with x and y features</h2>

In [8]:
df['dist'] = np.round(np.sqrt((df['x'])**2 + df['y']**2),0)  
df.head()

,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,outcome,dist
0,3,3,13.47,-11.22,1,0,2,70,1,18.0
1,3,4,9.48,14.22,3,0,2,55,0,17.0
2,2,4,29.43,-1.25,6,2,1,86,0,29.0
3,3,3,26.93,1.00,4,1,2,77,0,27.0
4,3,3,10.72,5.24,2,0,2,76,1,12.0


In [9]:
df['interveningOpponents'] = df['interveningOpponents'] + df['interveningTeammates']
df = df.drop(['interveningTeammates'],axis=1)
df.head()

,playType,bodyPart,x,y,interveningOpponents,interferenceOnShooter,minute,outcome,dist
0,3,3,13.47,-11.22,1,2,70,1,18.0
1,3,4,9.48,14.22,3,2,55,0,17.0
2,2,4,29.43,-1.25,8,1,86,0,29.0
3,3,3,26.93,1.00,5,2,77,0,27.0
4,3,3,10.72,5.24,2,2,76,1,12.0


<h2>Recognize zone with x and y</h2>

In [10]:
df['zone'] = 0
df['zone'][(df['x'] < 5.5) & ((df['y'] > -9.16) & (df['y'] < 9.16))] = 6
# dft['zone'][ 
#             ((dft['x'] < 5.5) & (((dft['y'] > 9.16) & (dft['y'] <20.16))|((dft['y'] < -9.16) & (dft['y'] > -20.16)))) & 
#             ((dft['x'] > 11)&(dft['x'] < 16.5) & (dft['y'] < 20.16) & (dft['y'] > -20.16))] = 5
df['zone'][(df['x'] < 30) & (df['x'] > 16.5) & (df['y'] > -20.16) & (df['y'] < 20.16)] = 4
df['zone'][(df['x'] < 45) & (df['x'] > 30) & (df['y'] > -20.16) & (df['y'] < 20.16)] = 3
df['zone'][(df['y'] < -20.16) | (df['y'] > 20.16)] = 2
df['zone'][df['x'] > 45] = 1
df['zone'][df['zone'] == 0] = 5
df.head()

C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3017229968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['zone'][(df['x'] < 5.5) & ((df['y'] > -9.16) & (df['y'] < 9.16))] = 6
C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3017229968.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['zone'][(df['x'] < 30) & (df['x'] > 16.5) & (df['y'] > -20.16) & (df['y'] < 20.16)] = 4
C:\Users\Dorsait.com\AppData\Local\Temp\ipykernel_22608\3017229968.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

,playType,bodyPart,x,y,interveningOpponents,interferenceOnShooter,minute,outcome,dist,zone
0,3,3,13.47,-11.22,1,2,70,1,18.0,5
1,3,4,9.48,14.22,3,2,55,0,17.0,5
2,2,4,29.43,-1.25,8,1,86,0,29.0,4
3,3,3,26.93,1.00,5,2,77,0,27.0,4
4,3,3,10.72,5.24,2,2,76,1,12.0,5


In [11]:
df['minute'] = (df['minute'] / 15) + 1

<h3>Drop x and y beacuse we have distance and zone id and they're more efficient than x and y</h3>

In [12]:
df = df.drop(['x','y'],axis=1)
df.head()

,playType,bodyPart,interveningOpponents,interferenceOnShooter,minute,outcome,dist,zone
0,3,3,1,2,5.666667,1,18.0,5
1,3,4,3,2,4.666667,0,17.0,5
2,2,4,8,1,6.733333,0,29.0,4
3,3,3,5,2,6.133333,0,27.0,4
4,3,3,2,2,6.066667,1,12.0,5


<h2>Analyze Np.nan values</h2>

In [13]:
df.isnull().sum()

playType                  0
bodyPart                  0
interveningOpponents      0
interferenceOnShooter    34
minute                    0
outcome                   0
dist                      0
zone                      0
dtype: int64

<h3>So we have 34 nan values in our dataset, it's not too much so we can drop these rows</h3>

In [14]:
df = df.dropna()
df.shape

(8891, 8)

<h2>Change type of our dataframe</h2>

In [15]:
df = df.astype({
    'playType' : np.int32,
    'bodyPart' : np.int32,
    'interferenceOnShooter' : np.int32,
    'outcome' : np.int32
})

In [16]:
Y = df.outcome
df = df.drop(['outcome'],axis=1)
X = df.copy()

In [17]:
# X.to_csv('x_data.csv')
# Y.to_csv('y_data.csv')

<h2>Select train and test data</h2>

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=10,test_size=0.2)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(7112, 7) (7112,)
(1779, 7) (1779,)


<h1>Building the models</h1>
<h2>First Model: Support Vector Machine</h2>

In [30]:
from sklearn.metrics import *
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [20]:
model = svm.SVC(kernel = "sigmoid",C = 1.0)
model.fit(x_train,y_train)

SVC(kernel='sigmoid')

In [21]:
y_pred = model.predict(x_test)

In [22]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[1379  192]
 [ 199    9]]


In [23]:
print('Trp Metric:',cm[0,0]/ (cm[0,0] + cm[0,1]) * 100)

Trp Metric: 87.77848504137492



<h2>Second model: Deep neural network using Keras</h2>

In [20]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras import Sequential

In [23]:
model = Sequential()
model.add(Dense(30,input_shape=(7,)))
model.add(Dense(50,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['accuracy']
)

In [24]:
model.fit(x_train,y_train,epochs=50,batch_size=32,validation_split=0.15)

Epoch 1/50
189/189 [==============================] - 1s 3ms/step - loss: 0.1082 - accuracy: 0.8655 - val_loss: 0.0973 - val_accuracy: 0.8866
Epoch 2/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0935 - accuracy: 0.8792 - val_loss: 0.0912 - val_accuracy: 0.8847
Epoch 3/50
189/189 [==============================] - 1s 3ms/step - loss: 0.0921 - accuracy: 0.8822 - val_loss: 0.0913 - val_accuracy: 0.8838
Epoch 4/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0923 - accuracy: 0.8827 - val_loss: 0.0932 - val_accuracy: 0.8828
Epoch 5/50
189/189 [==============================] - 1s 3ms/step - loss: 0.0913 - accuracy: 0.8834 - val_loss: 0.0926 - val_accuracy: 0.8866
Epoch 6/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0908 - accuracy: 0.8842 - val_loss: 0.0892 - val_accuracy: 0.8810
Epoch 7/50
189/189 [==============================] - 0s 2ms/step - loss: 0.0903 - accuracy: 0.8850 - val_loss: 0.0900 - val_accuracy: 0.8885
Epoch 

In [25]:
_,accuracy_score = model.evaluate(x_train,y_train,verbose=0)

In [26]:
y_pred = model.predict(x_test)

56/56 [==============================] - 0s 1ms/step


In [27]:
y_pred = y_pred.reshape(-1,1)
y_pred.shape

(1779, 1)

In [28]:
for i in range(y_pred.shape[0]):
    if y_pred[i] >= 0.5 :
        y_pred[i] = 1
    elif y_pred[i] < 0.5:
        y_pred[i] = 0

In [31]:
cm = confusion_matrix(y_test,y_pred)
print('Trp Metric:',cm[0,0]/(cm[0,0] + cm[0,1]) * 100)

Trp Metric: 99.04519414385742
